## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ["Age", "Education", "HourlyRate", "JobInvolvement", "StockOptionLevel", "TotalWorkingYears", "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion", "NumCompaniesWorked"]


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                        int64
Education                  int64
HourlyRate                 int64
JobInvolvement             int64
StockOptionLevel           int64
TotalWorkingYears          int64
YearsAtCompany             int64
YearsInCurrentRole         int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [8]:
from sklearn.preprocessing import OneHotEncoder
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[["Department"]])
department_test_encoded = department_encoder.transform(y_test[["Department"]])

print(department_train_encoded)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[["Attrition"]])
attrition_test_encoded = attrition_encoder.transform(y_test[["Attrition"]])

print(attrition_train_encoded)


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [10]:
print(X_train.shape)

(1176, 10)


In [11]:
# Find the number of columns in the X training data
input_nodes = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name="input_layer")

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation="relu", name="shared1")(input_layer)
shared_layer2 = layers.Dense(128, activation="relu", name="shared2")(shared_layer1)


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32,
                                       activation="relu",
                                       name="department_hidden")(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(3,
                                       activation="softmax",
                                       name="department_output")(department_hidden_layer)


In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32,
                                      activation="relu",
                                      name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(2,
                                      activation="sigmoid",
                                      name="attrition_output")(attrition_hidden_layer)



In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[
    department_output_layer,
    attrition_output_layer
])

# Compile the model
model.compile(optimizer="adam",
              loss={
                  "department_output": "binary_crossentropy",
                  "attrition_output": "binary_crossentropy"
              },
              metrics={
                  "department_output": "accuracy",
                  "attrition_output": "accuracy"
              })

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'department_output': department_train_encoded,
        'attrition_output': attrition_train_encoded
    },
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - attrition_output_accuracy: 0.7512 - department_output_accuracy: 0.6101 - loss: 1.2156 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.0355
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8512 - department_output_accuracy: 0.6699 - loss: 0.9201 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 0.9904
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8343 - department_output_accuracy: 0.6504 - loss: 0.9091 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 0.9926
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8394 - department_output_accuracy: 0.6713 - loss: 0.8845 - val_attrition_output_accuracy: 0.8220 - val_department_output_accuracy: 0.6314 - val_loss: 0.9874
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [16]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {
        "department_output": department_test_encoded,
        "attrition_output": attrition_test_encoded
    })

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8472 - department_output_accuracy: 0.6493 - loss: 0.9121 


In [17]:
print(results)

[0.8571272492408752, 0.8673469424247742, 0.6666666865348816]


In [18]:
# Print the accuracy for both department and attrition
pred_categories = ["Department", "Attrition"]
for i, cat in enumerate(pred_categories):
    print(f"{cat} predictions accuracy: {results[i+1]}")

Department predictions accuracy: 0.8673469424247742
Attrition predictions accuracy: 0.6666666865348816


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is generally a good metric to go by, but it is not always the best. There are many factors that could go into a model that could skew an accuracy score and you generally need to look at multiple points as what could be found in a classification report, as one example.
2. For Attrition I went with sigmoid as an employee either left their position or stayed which is a binary outcome. For Department, since there were multiple classifications, softmax seemed like a good activation function.
3. Using different features to base the model on, possibly do some feature engineering. Adding more layers or nodes to the hidden layers could also enhance the overall probabilities for the model.